<a href="https://colab.research.google.com/github/JuanFelipePosada/python-basico_/blob/main/punto_1_2_parcial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install -U yt-dlp
!pip install yt-dlp soundfile matplotlib streamlit numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.

In [2]:
%%writefile app.py
import requests
from io import BytesIO
import yt_dlp as youtube_dl
import subprocess
import os
import shutil
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tempfile
from scipy.fftpack import fft
from scipy.signal import square
from scipy.interpolate import interp1d

# Configuración inicial de la página
st.set_page_config(page_title="Parcial 2 - Fourier SyS", page_icon="📊", layout="wide")

# Barra lateral para seleccionar el punto del parcial y mostrar información general
with st.sidebar:
    st.image("https://lh4.googleusercontent.com/proxy/WNtyuTbDjnnITJFxg1dlI63L0jfIMRf0CIKg75VavFd3ameUuokpEiXIZvafO0UbA3rGKkhjDZ2HFtRWcGiPIn7Syd37PqnCrQuXFNHguRRPYm__safRJi9Q", width=150)
    st.title("Parcial 2 - Fourier SyS")
    st.subheader("Universidad Nacional de Colombia - Sede Manizales")
    st.write("**Señales y Sistemas - 2024-1**")
    st.write("**Integrantes:**")
    st.write("- Juan Felipe Posada Niasa")
    st.write("- Juan Jose Montilla Bastidas")

    # Menú desplegable para seleccionar el punto del parcial
    punto = st.selectbox("Selecciona el punto del parcial",
                         ("Punto 1: Expresión del Espectro de Fourier",
                          "Punto 2: Simulación y Espectro de Fourier",
                          "Punto 3: Modulación en Amplitud (AM) con Audio",
                          "Punto 4: THD y Factor de Potencia"))

# Mostrar información general del punto seleccionado
if punto == "Punto 1: Expresión del Espectro de Fourier":

        st.write("### Expresión del Espectro de Fourier (Forma Exponencial y Trigonométrica) ")

        # Desarrollo de la señal
        st.write("#### Desarrollo:")
        st.write("Dada la señal original:")
        st.latex(r"x(t) = |A \sin(2\pi F_o t)|^2 = A^2 \sin^2(2\pi F_o t) = A^2 \left( \frac{1}{2} - \frac{\cos(2\pi 2 F_o t)}{2} \right)")

        st.write("Aplicando la serie trigonométrica:")
        st.latex(r"x(t) = a_0 + \sum_{n=-N}^N a_n \cos(n\omega_0 t) + b_n \sin(n\omega_0 t)")
        st.write("Como la señal es par, \(b_n = 0\) para todos los \(n\). Los coeficientes son:")
        st.latex(r"a_0 = \frac{A^2}{2}")
        st.latex(r"a_2 = -\frac{A^2}{2}")
        st.latex(r"a_n = 0 \quad \text{para} \quad n \neq 2")

        st.write("Para la serie exponencial compleja:")
        st.latex(r"x(t) = \sum_{n=-N}^{N} c_n e^{j n \omega_0 t}")
        st.write("Donde:")
        st.latex(r"c_0 = \frac{A^2}{2}")
        st.latex(r"c_2 = -\frac{A^2}{4}")
        st.latex(r"c_{-2} = \frac{A^2}{4}")

        st.write("## Parametros para de la señal")

        A = st.slider("Amplitud de señal(A)", 1, 11, 0)
        F_o= st.slider("Frecuencia de señal original(F)", 0, 120  , step=10)
        ti= -1/ (2 * F_o)
        tf= 1/ (2 * F_o)
        T = 1 / F_o
        Nm= 1000
        tv = np.linspace(ti, tf, Nm)
        x = np.abs(A * np.sin(2 * np.pi * F_o * tv))**2

        # Graficar la señal
        st.write("### Gráfico de la señal $|A\\sin(2\\pi F_o t)|^2$")
        fig, ax = plt.subplots()
        ax.plot(tv, x)
        ax.set_xlabel('Tiempo [s]')
        ax.set_ylabel('$x(t)$')
        ax.grid(True)
        st.pyplot(fig)

#######################################################################################
elif punto == "Punto 2: Simulación y Espectro de Fourier":
  st.write("### Simulacion y Espectros de Fourier")
  st.write("#### ahora graficamos los espectros, fase y magnitud de la serie de fourier que hallamos en el punto 1")
  # valores constantes
  N = 5
  A = st.slider("Amplitud de señal(A)", 1, 11, 0)
  F_o= st.slider("Frecuencia de señal original(F)", 0, 120  , step=10)

  ti= -1/ (2 * F_o)
  tf= 1/ (2 * F_o)
  T = 1 / F_o
  Nm= 1000
  tv = np.linspace(ti, tf, Nm)
  T = 1 / F_o
  Wo = 2 * np.pi / T
  x = np.abs(A * np.sin(2 * np.pi * F_o * tv))**2

  phin = np.zeros((len(tv), 2 * N + 1), dtype=np.complex_)
  for n in range(-N, N + 1):
      phin[:, n + N] = np.exp(1j * n * Wo * tv)

  # Mostrar las bases de Fourier una a la vez
  n_base= st.slider("Selecciona el armonico a graficar", -N, N,0)


  fig, ax = plt.subplots()
  ax.plot(tv, np.real(phin[:, n_base + N]), label=f"$Re(\\phi_{n_base}(t))$")
  ax.plot(tv, np.imag(phin[:, n_base + N]), label=f"$Im(\\phi_{n_base}(t))$")
  ax.set_xlabel("t [s]")
  ax.set_ylabel("espectros de Fourier")
  ax.legend()
  ax.grid(True)
  st.pyplot(fig)

  cn = np.zeros(2 * N + 1, dtype=np.complex_)
  cn[N] = A**2 / 2  # Nivel DC
  cn[N - 2] = -A**2 / 4  # Armónico -2
  cn[N + 2] = -A**2 / 4  # Armónico 2
  nv = np.linspace(-N, N, 2 * N + 1)  # Vector de armónicos
  # Gráficos del espectro de Fourier
  st.write("### Espectro de Fourier:")
  fig, axs = plt.subplots(2, 2, figsize=(10, 10))

  # Real de Cn
  axs[0, 0].stem(nv, np.real(cn), 'r')
  axs[0, 0].set_xlabel(r'$n\omega_0$ [rad/s]')
  axs[0, 0].set_ylabel(r'$Re\{C_n\}$')
  axs[0, 0].grid()

  # Imaginario de Cn
  axs[0, 1].stem(nv, np.imag(cn), 'r')
  axs[0, 1].set_xlabel(r'$n\omega_0$ [rad/s]')
  axs[0, 1].set_ylabel(r'$Im\{C_n\}$')
  axs[0, 1].grid()

  # Magnitud de Cn
  axs[1, 0].stem(nv, abs(cn), 'r')
  axs[1, 0].set_xlabel(r'$n\omega_0$ [rad/s]')
  axs[1, 0].set_ylabel(r'$|C_n|$')
  axs[1, 0].grid()

  # Fase de Cn
  axs[1, 1].stem(nv, np.angle(cn), 'r')
  axs[1, 1].set_xlabel(r'$n\omega_0$ [rad/s]')
  axs[1, 1].set_ylabel(r'$\langle C_n$')
  axs[1, 1].grid()

  fig.tight_layout()
  st.pyplot(fig)

  # Graficar la reconstrucción de la señal
  st.write("### Reconstrucción de la señal y error relativo")

  def pltest(Na=1):
      ind = np.arange(N - Na, N + Na + 1)

      xe = phin[:, ind].dot(cn[ind])  # Señal reconstruida
      er = error_relativo = np.linalg.norm(x - xe) / np.linalg.norm(x)# Error relativo

      # Graficar la señal reconstruida y la original
      fig, ax = plt.subplots()
      ax.plot(tv, xe, label='$x_e(t)$', color='b')
      ax.plot(tv, x, label='$x(t)$', color='r')
      ax.set_title(f'Error Relativo = {100 * er:.2f}%')
      ax.set_xlabel('t [s]')
      ax.set_ylabel('x(t)')
      ax.grid(True)
      ax.legend()
      st.pyplot(fig)


  # Interfaz interactiva para cambiar Na
  st.write("Ajusta el número de armónicos \(N\) utilizados para la reconstrucción:")
  Na = st.slider('Número de armónicos', 1, N, 1)
  pltest(Na)

  # Frecuencias angulares
  frequencies = nv * Wo

  # Calcular la magnitud y la fase a partir de cn
  magnitudes = 20 * np.log10(np.abs(cn))
  phases = np.angle(cn, deg=True)

  # Filtrar frecuencias positivas (evitando el DC y negativos)
  positive_indices = np.where(frequencies > 0)[0]
  positive_frequencies = frequencies[positive_indices]
  positive_magnitudes = magnitudes[positive_indices]
  positive_phases = phases[positive_indices]

  # Interpolación de magnitudes y fases para las frecuencias logarítmicas
  log_freq = np.logspace(np.log10(np.min(positive_frequencies)), np.log10(np.max(positive_frequencies)), num=500)
  mag_interp = interp1d(positive_frequencies, positive_magnitudes, kind='linear', fill_value='extrapolate')
  phase_interp = interp1d(positive_frequencies, positive_phases, kind='linear', fill_value='extrapolate')

  # Magnitud y fase interpoladas en frecuencias logarítmicas
  log_magnitudes = mag_interp(log_freq)
  log_phases = phase_interp(log_freq)

  # Graficar el Diagrama de Bode
  st.write("### Diagrama de Bode")

  fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

  # Diagrama de Bode - Magnitud
  ax1.semilogx(log_freq, log_magnitudes, 'b-', marker='o')  # Magnitud en dB
  ax1.set_title('Diagrama de Bode')
  ax1.set_xlabel('Frecuencia [rad/s]')
  ax1.set_ylabel('Magnitud [dB]')
  ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

  # Diagrama de Bode - Fase
  ax2.semilogx(log_freq, log_phases, 'r-', marker='o')  # Fase en grados
  ax2.set_xlabel('Frecuencia [rad/s]')
  ax2.set_ylabel('Fase [grados]')
  ax2.grid(True, which='both', linestyle='--', linewidth=0.5)

  plt.tight_layout()
  st.pyplot(fig)





Writing app.py


In [3]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFiUsTGsQClQyxWz9mY9rLIup_7vUusFxPVHjfdMEa1CryT"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")

nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://3c68-34-106-39-30.ngrok-free.app" -> "http://localhost:5011"
